# IntelHealth - SFT 数据集生成（Colab）

说明：一次只生成一个数据集。建议先用 `--num_samples 50` 做小样测试，再放大。


In [ ]:
# Clone repo (skip if already cloned)
import os
if not os.path.exists('Intel_Health'):
    !git clone https://github.com/DemonRain7/Intel_Health.git
%cd Intel_Health

In [ ]:
# 安装依赖
!pip -q install openai tiktoken jsonschema loguru tqdm

## 1) 设置环境变量
把你的 OpenAI Key 填进下面（只在 Colab 运行时生效）。


In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = (userdata.get("OPENAI_API_KEY") or os.environ.get("OPENAI_API_KEY", "")).strip()
# 如果你有代理或自建 API 网关：
# os.environ["OPENAI_BASE_URL"] = ""


## 2) Preprocess / Symptom Normalizer（多阶段生成）

- **seed 模型**：`gpt-5-mini` 生成主诉 + 结构化字段
- **final 模型**：`gpt-4.1` 生成规范输出


In [ ]:
# 生成 preprocess 数据（一次只跑这一条）
!python pipeline/sft/build_preprocess_sft.py \
  --num_samples 1500 \
  --output datasets/agent_sft/symptom_normalizer/preprocess_sft_data.jsonl \
  --seed_model gpt-5-mini \
  --final_model gpt-4.1 \
  --output_format conversation \
  --resume \
  --price_in_seed 0.25 --price_out_seed 2.00 \
  --price_in_final 2.00 --price_out_final 8.00

## 3) 其它需要 SFT 的 Agent（逐个生成）

- `symptom_quality_grader`
- `rag_relevance_grader`
- `drug_evidence_grader`

一次只运行其中一个。


In [ ]:
# symptom_quality_grader
!python pipeline/sft/generate_sft_data.py \
  --agent symptom_quality_grader \
  --num_samples 1500 \
  --output_dir datasets/agent_sft/symptom_quality_grader \
  --model gpt-4.1 \
  --output_format conversation \
  --resume

In [ ]:
# rag_relevance_grader
!python pipeline/sft/generate_sft_data.py \
  --agent rag_relevance_grader \
  --num_samples 1500 \
  --output_dir datasets/agent_sft/rag_relevance_grader \
  --model gpt-4.1 \
  --output_format conversation \
  --resume

In [ ]:
# drug_evidence_grader
!python pipeline/sft/generate_sft_data.py \
  --agent drug_evidence_grader \
  --num_samples 1500 \
  --output_dir datasets/agent_sft/drug_evidence_grader \
  --model gpt-4.1 \
  --output_format conversation \
  --resume

## 4) Diagnosis DAPT（从 train_sft.jsonl 转换）

该数据集用于诊断模型的 **领域预训练 / LoRA**，不做结构化输出 SFT。


In [ ]:
!python pipeline/sft/build_diagnosis_dapt.py \
  --input datasets/agent_sft/train_sft.jsonl \
  --output datasets/agent_sft/diagnosis_generator/diagnosis_dapt.jsonl \
  --resume